# Tuned - Inference

## Install Dependencies

In [3]:
GDRIVE_BASE = 'drive/MyDrive/MIDS/w266/project/'

!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 28.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00


## Connect to Google Drive
We will be loading data from google drive and also save trained models to google drive. So lets mount google drive.

In [13]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0, GDRIVE_BASE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports and Constants

In [22]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, OPTForCausalLM
import torch
import transformers
from collections import deque
from tqdm.notebook import tqdm
import common

print(f'common.__version__: {common.__version__}')

transformers.logging.set_verbosity_error()
tuning_configs = common.create_configs(GDRIVE_BASE, None, None, None, None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
common.__version__: 1.4


# Story Bot

In [15]:
class StoryBot:
  """Class to mimic a bot that continues the story."""
  def __init__(self, inferencer, n_iters=20, lines_to_use=1):
    """
      Creates the interactive story bot.
      inferencer - class to use for generating lines of the story.
      n_iters - number of iterations to do for story generation.
      lines_to_use - The number of lines from the story to use as context for 
                     generating the next line.
    """
    self._n_iters = n_iters
    self.lines_to_use = lines_to_use
    self.inferencer = inferencer
    self.re_init()

  def re_init(self):
    self.story = []
    # Initialize queue to hold just the last "lines_to_use" lines of story.
    self.context_lines = deque([], self.lines_to_use)

  def display_line_choices(self, output_lines):
    print('Choose the line of your choice:')
    for i, line in enumerate(output_lines):
      print(f'{i}:', line)
    print(f'{i+1}: Regenerate')
    print(f'{i+2}: End')

  def get_user_choice(self):
    output_lines = self.inferencer(self.context_lines)
    if len(output_lines) > 1:
      self.display_line_choices(output_lines)
      user_opt = -1
      while user_opt == -1:
        try:
            user_input = input('Input the number of your choice (or ): ')
            user_opt = int(user_input)
            if user_opt < len(output_lines):
              return output_lines[user_opt]
            elif user_opt == len(output_lines):
              return 'regenerate'
            elif user_opt == len(output_lines) + 1:
              return 'end'
        except ValueError:
            user_opt = -1
    else:
      return output_lines[0]

  def print_story(self):
    for i, line in enumerate(self.story):
      if i%2 == 0:
        print(f'User: {line}') 
      else:
        print(f'Generated: {line}') 

  def __call__(self):
    print('*'*50)
    print('Welcome to StoryBot!\n')
    print('This program simulates an MMS kind of interaction with a bot to create a story sequentially.')
    print('When the prompt appears below, start typing as if it were the input on your mobile.')
    print('Enter end to end the story and restart to restart.') 
    print('*'*50, '\n')
    restart = False
    i = 0
    while i < self._n_iters:
      if i > 0:
        print('The story so far:')
        self.print_story()
      i = i + 1
      # get the sentence from the user
      sentence_in = input('Enter next line (or end): ').strip()
      # accomodate special prompts
      if sentence_in == 'end':
        break
      if sentence_in == 'restart':
        i = 0
        self.re_init()
        continue
      self.context_lines.append(sentence_in)
      self.story.append(sentence_in)
      output = 'regenerate'
      while output == 'regenerate':
        output = self.get_user_choice()
      if output == 'end':
        break
      self.context_lines.append(output)
      self.story.append(output)

    print()
    print('\n======== Final story: =========\n')
    self.print_story()



In [ ]:
def run_story_bot(config, device):
  tokenizer = AutoTokenizer.from_pretrained(config.model_name)
  if config.model_family == 't5':
    model = T5ForConditionalGeneration.from_pretrained(config.tuned_model_path).to(device)
    inferencer = common.T5Inferencer(device, model, tokenizer, prompt=config.prompt)
  else:
    model = OPTForCausalLM.from_pretrained(config.model_name).to(device)
    inferencer = common.OptInferencer(device, model, tokenizer)
  story_bot = StoryBot(inferencer, n_iters=1, lines_to_use=1)
  story_bot()


In [ ]:
# Run story bot on t5 s1
run_story_bot(tuning_configs['t5_s1'], device)

**************************************************
Welcome to StoryBot!

This program simulates an MMS kind of interaction with a bot to create a story sequentially.
When the prompt appears below, start typing as if it were the input on your mobile.
Enter end to end the story and restart to restart.
************************************************** 

Enter next line (or end): The little girl felt very lonely and scared.
Choose the line of your choice:
0: "It's a good thing," said she.
1: "It's a very pleasant place," said the little girl.
2: "It's all right," she said.
3: Regenerate
4: End
Input the number of your choice (or ): 2


======== Final story: =========

User: The little girl felt very lonely and scared.
Generated: "It's all right," she said.


In [ ]:
# Run story bot on opt s2
run_story_bot(tuning_configs['opt_s2'], device)

**************************************************
Welcome to StoryBot!

This program simulates an MMS kind of interaction with a bot to create a story sequentially.
When the prompt appears below, start typing as if it were the input on your mobile.
Enter end to end the story and restart to restart.
************************************************** 

Enter next line (or end): The little girl felt very lonely and scared.
Choose the line of your choice:
0: The little girl felt very lonely and scared. She was crying and crying, and her parents were crying with her, but she didn't want to talk to them because she couldn't say anything, so she just sat there and cried for a
1: The little girl felt very lonely and scared. She wanted to go to school, but she didn't know where to find a teacher to help her. The teacher told her that she had to get a diploma to be accepted into a school.
2: The little girl felt very lonely and scared.  She was called to the hospital to be checked out, but she 

# Batch Inferencing

In [25]:
def max_score_index(scores):
  max_score = scores[0]
  max_score_index = 0
  for i, score in enumerate(scores):
    if score > max_score:
      # New max score
      max_score = score
      max_score_index = i
  return max_score_index

def generate_next_line(config, line, device, num_sequences=5):
  tokenizer = AutoTokenizer.from_pretrained(config.model_name)
  test_inputs = tokenizer([config.prompt + line], return_tensors='pt')
  input_ids = test_inputs['input_ids'].to(device)
  if config.model_family == 't5':
    if config.tuned:
      model = T5ForConditionalGeneration.from_pretrained(config.tuned_model_path).to(device)
    else:
      model = T5ForConditionalGeneration.from_pretrained(config.model_name).to(device)
    outputs = model.generate(
        input_ids,
        num_beams=num_sequences,
        no_repeat_ngram_size=2,
        num_return_sequences=num_sequences,
        max_new_tokens=100,
        do_sample=True,
        top_k=0,
        return_dict_in_generate=True,
        output_scores=True,
        renormalize_logits=True,
        # normalize_logits=True,
      )
  else:
    if config.tuned:
      model = OPTForCausalLM.from_pretrained(config.tuned_model_path).to(device)  
    else:
      model = OPTForCausalLM.from_pretrained(config.model_name).to(device)  
    outputs = model.generate(
      input_ids,
      num_beams=num_sequences,
      no_repeat_ngram_size=2,
      num_return_sequences=num_sequences,
      max_length = 50,
      do_sample=True,
      top_k=0,
      early_stopping=True,
      return_dict_in_generate=True,
      output_scores=True,
      renormalize_logits=True
    )

  scores = (-1 * outputs.sequences_scores).tolist()
  test_output_ids = outputs.sequences
  decoded = [tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False).replace('\n', ' ') for out_ids in test_output_ids]

  result = [[config.name, config.model_name, config.tuned, config.dataset, line, score, output, False] for score, output in zip(scores, decoded)]
  result[max_score_index(scores)][-1] = True
  return result

def generate_for(config, device, first_lines):
  print('*' *50)
  print(f'Evaluating {config.model_name} tuned on {config.dataset} dataset')
  return generate_next_line(config, first_lines, device)



In [33]:
first_lines = ['Lara felt very sad and scared.', 'All the dragons of the world lived on one mountain called the dragon mountain.']
results = []

with tqdm(total=len(tuning_configs) * len(first_lines), unit='text genaration', unit_scale=True) as pbar:
  for line in first_lines:
    for name, config in tuning_configs.items():
      pbar.set_postfix(model=name, prompt=line[0:10] + '...', refresh=True)
      results.extend(generate_next_line(config, line, device))
      pbar.update(1)
  df_results = pd.DataFrame(results, columns=['name', 'model', 'tuned', 'dataset', 'prompt', 'score', 'top_scored', 'generated'])
  

  0%|          | 0.00/12.0 [00:00<?, ?text genaration/s]

In [34]:
df_results

,name,model,tuned,dataset,prompt,score,top_scored,generated
0,t5_s1,google/t5-v1_1-base,True,s1,Lara felt very sad and scared.,4.857201e-05,"""What do you mean?"" cried she, turning to him.",False
1,t5_s1,google/t5-v1_1-base,True,s1,Lara felt very sad and scared.,5.416459e-05,"""Well,"" said a maid, ""you're hardly able to he...",False
2,t5_s1,google/t5-v1_1-base,True,s1,Lara felt very sad and scared.,6.966857e-05,"""What do you mean?"" asked the young man.",True
3,t5_s1,google/t5-v1_1-base,True,s1,Lara felt very sad and scared.,2.838925e-05,"“There is nothing to do with it,” said she, an...",False
4,t5_s1,google/t5-v1_1-base,True,s1,Lara felt very sad and scared.,3.685435e-05,"""I have a great heart,"" she cried.",False
5,t5_s2,google/t5-v1_1-base,True,s2,Lara felt very sad and scared.,1.639245e-05,"Then she ran to her mother, and said, “If I di...",False
6,t5_s2,google/t5-v1_1-base,True,s2,Lara felt very sad and scared.,6.261440e-05,she could not understand why she was afraid of...,False
7,t5_s2,google/t5-v1_1-base,True,s2,Lara felt very sad and scared.,3.400600e-05,She was not a nice girl; but if she could find...,False
8,t5_s2,google/t5-v1_1-base,True,s2,Lara felt very sad and scared.,2.821932e-05,Her arms were so small and her head so heavy t...,False
9,t5_s2,google/t5-v1_1-base,True,s2,Lara felt very sad and scared.,9.815981e-05,She looked very pale.,True
